<a href="https://colab.research.google.com/github/sai-phani255/Book-Recommendation-System/blob/master/Social_Comment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**CONTENT BASED POST RECOMMENDATION SYSTEM**

This system is entirely different from the Content Based Movie Recommendation System.In the Movie Recommendation system we only have to recommend the names of the movies.
So for movie recommendation system we have to give the exact name of the movie.

But in **Post Recommendation System**, posts related to the **given word** are also to be recommended by the system, even if the word is not present in the dataset.So in this type of recommendation system, each and every word plays an important role.


So I tried to implement this idea. For that I have used Lemmatization, so that we can get the exact root word.

#**DATA COLLECTION**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

views_data = pd.read_csv('/content/drive/MyDrive/MLdata/views.csv')
users_data = pd.read_csv('/content/drive/MyDrive/MLdata/users.csv')
posts_data = pd.read_csv('/content/drive/MyDrive/MLdata/posts.csv')

In [4]:
print("Views Data",views_data.shape)
print("Users Data",users_data.shape)
print("Posts Data",posts_data.shape)

Views Data (1449, 3)
Users Data (118, 4)
Posts Data (493, 4)


In [5]:
views_data.head()

,user_id,post_id,timestamp
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z
1,5ed3748576027d35905ccaab,5ed4cbadbd514d602c1531a6,2020-06-01T09:39:20.021Z
2,5ed0defa76027d35905cc2de,5eac305f10426255a7aa9dd3,2020-06-01T08:12:42.682Z
3,5ed0defa76027d35905cc2de,5ed1ff0276027d35905cc60d,2020-06-01T08:10:23.880Z
4,5ed0defa76027d35905cc2de,5ed3820f76027d35905ccac8,2020-06-01T08:08:54.124Z


In [6]:
views_data.isnull().sum()

user_id      0
post_id      0
timestamp    0
dtype: int64

In [7]:
users_data.head()

,_id,name,gender,academics
0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate
1,5d610ae1653a331687083239,Gaurav Sharma,male,graduate
2,5d618359fc5fcf3bdd9a0910,Akshay Mishra,male,undergraduate
3,5d6d2bb87fa40e1417a49315,Saksham Mathur,male,undergraduate
4,5d7c994d5720533e15c3b1e9,Varun Chowhan,male,undergraduate


In [8]:
users_data.isnull().sum()

_id          0
name         0
gender       0
academics    0
dtype: int64

In [9]:
posts_data.head()

,_id,title,category,post_type
0,5d62abaa65218653a132c956,hello there,Plant Biotechnology,blog
1,5d6d39567fa40e1417a4931c,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog
2,5d7d23315720533e15c3b1ee,What is an Operating System ?,Operating Systems,blog
3,5d7d405e5720533e15c3b1f3,Lord Shiva,Drawings,artwork
4,5d80dfbc6c53455f896e600e,How Competition law evolved?,Competition Laws,blog


In [10]:
posts_data.isnull().sum()

_id            0
title          0
category      28
 post_type     0
dtype: int64

In [11]:
views_data = views_data.drop(['timestamp'],axis=1)

In [12]:
posts_data.rename(columns={'_id': 'post_id', ' post_type': 'post_type'}, inplace=True)
users_data.rename(columns={'_id': 'user_id'}, inplace=True)
posts_data.category = posts_data.category.fillna('')

In [13]:
Req_data = pd.merge(views_data,users_data ,on ='user_id')
Req_data = pd.merge(Req_data, posts_data, on='post_id')

In [14]:
Req_data.head()

,user_id,post_id,name,gender,academics,title,category,post_type
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,Niriksha Sharma,female,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
1,5d7c994d5720533e15c3b1e9,5ec821ddec493f4a2655889e,Varun Chowhan,male,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
2,5e5af599d701ab08af792b63,5ec821ddec493f4a2655889e,Ilupeju Ayokunnumi,female,graduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
3,5de50d768eab6401affbb135,5ec821ddec493f4a2655889e,thesocialcomment,male,graduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
4,5deeef6142a8854bf6eabab9,5ec821ddec493f4a2655889e,siddharth saxena,male,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork


In [15]:
Req_data.shape

(1447, 8)

In [16]:
Req_data.isnull().sum()

user_id      0
post_id      0
name         0
gender       0
academics    0
title        0
category     0
post_type    0
dtype: int64

In [17]:
# Replacing the null values with space because category is textual data
Req_data.category = Req_data.category.fillna('')

In [18]:
Req_data.isnull().sum()

user_id      0
post_id      0
name         0
gender       0
academics    0
title        0
category     0
post_type    0
dtype: int64

#**DATA PREPOCESSING**

In [19]:
Req_data.head()

,user_id,post_id,name,gender,academics,title,category,post_type
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,Niriksha Sharma,female,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
1,5d7c994d5720533e15c3b1e9,5ec821ddec493f4a2655889e,Varun Chowhan,male,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
2,5e5af599d701ab08af792b63,5ec821ddec493f4a2655889e,Ilupeju Ayokunnumi,female,graduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
3,5de50d768eab6401affbb135,5ec821ddec493f4a2655889e,thesocialcomment,male,graduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
4,5deeef6142a8854bf6eabab9,5ec821ddec493f4a2655889e,siddharth saxena,male,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork


Now there is no use user_id , post_id and name. So droping those columns


In [20]:
Req_data = Req_data.drop(['user_id','post_id','name'],axis=1)

In [21]:
Req_data.head()

,gender,academics,title,category,post_type
0,female,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
1,male,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
2,female,graduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
3,male,graduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
4,male,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork


In [22]:
Req_data['academics'].unique()

array(['undergraduate', 'graduate', 'undefined'], dtype=object)

In [23]:
Req_data['gender'].unique()

array(['female', 'male', 'undefined'], dtype=object)

In [24]:
Req_data.groupby('gender').count()

,academics,title,category,post_type
gender,,,,
female,444,444,444,444
male,997,997,997,997
undefined,6,6,6,6


In [25]:
Req_data.groupby('academics').count()

,gender,title,category,post_type
academics,,,,
graduate,382,382,382,382
undefined,6,6,6,6
undergraduate,1059,1059,1059,1059


In [26]:
len(Req_data['category'].unique())

232

Category feature is textual and categorical.

Title feature is textual and is not categorical.


In [27]:
Req_data =  Req_data.rename(index=str, columns={" post_type":"post_type"})

In [28]:
# Relpacing the undefined in gender feature to male. Because it is the most occured in the dataset.
Req_data.gender = Req_data.gender.replace('undefined','male')

In [29]:
Req_data.groupby('gender').count()

,academics,title,category,post_type
gender,,,,
female,444,444,444,444
male,1003,1003,1003,1003


In [30]:
# Replacing the undefined in the academics feature to undergraduate.Because it is the most occuered in that feature.
Req_data.academics = Req_data.academics.replace('undefined','undergraduate')

In [31]:
Req_data.groupby('academics').count()

,gender,title,category,post_type
academics,,,,
graduate,382,382,382,382
undergraduate,1065,1065,1065,1065


In [32]:
# Lowering the data Because machine treats the lower case and upper case letters differently.

for column in Req_data:
  Req_data[column] = Req_data[column].str.lower()

In [33]:
Req_data.head()

,gender,academics,title,category,post_type
0,female,undergraduate,save earth.,visual arts|graphic design|artistic design|gra...,artwork
1,male,undergraduate,save earth.,visual arts|graphic design|artistic design|gra...,artwork
2,female,graduate,save earth.,visual arts|graphic design|artistic design|gra...,artwork
3,male,graduate,save earth.,visual arts|graphic design|artistic design|gra...,artwork
4,male,undergraduate,save earth.,visual arts|graphic design|artistic design|gra...,artwork


# **DATA CLEANING**

In [34]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [35]:
from nltk.corpus import stopwords

import string
import re

stop_words = stopwords.words("english")

def CleanText(x):
  x = x.lower()
  x = ' '.join([word for word in x.split(' ') if word not in stop_words])
  x = x.encode('ascii', 'ignore').decode()
  x = re.sub(r'https*\S+', ' ', x)
  x = re.sub(r'@\S+', ' ', x)
  x = re.sub(r'#\S+', ' ', x)
  x = re.sub(r'\'\w+', '', x)
  x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
  x = re.sub(r'\w*\d+\w*', '', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x

In [36]:
Req_data["gender"] = Req_data.gender.apply(CleanText)
Req_data["academics"] = Req_data.academics.apply(CleanText)
Req_data["title"] = Req_data.title.apply(CleanText)
Req_data["category"] = Req_data.category.apply(CleanText)
Req_data["post_type"] = Req_data.post_type.apply(CleanText)

In [37]:
Req_data.head()

,gender,academics,title,category,post_type
0,female,undergraduate,save earth,visual arts graphic design artistic design gra...,artwork
1,male,undergraduate,save earth,visual arts graphic design artistic design gra...,artwork
2,female,graduate,save earth,visual arts graphic design artistic design gra...,artwork
3,male,graduate,save earth,visual arts graphic design artistic design gra...,artwork
4,male,undergraduate,save earth,visual arts graphic design artistic design gra...,artwork


Here the data we have got is clean data that is, the data in the features only contain the alphabet.

#Performing Lemmatization.

In [38]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

def Lemmatize(x):
  words = nltk.word_tokenize(x)
  #print(words)
  words = [lemmatizer.lemmatize(word) for word in words if word not in set(stopwords.words('english')) ]
  sent = " ".join(words)
  return sent

In [39]:
Req_data["gender"] = Req_data.gender.apply(Lemmatize)
Req_data["academics"] = Req_data.academics.apply(Lemmatize)
Req_data["title"] = Req_data.title.apply(Lemmatize)
Req_data["category"] = Req_data.category.apply(Lemmatize)
Req_data["post_type"] = Req_data.post_type.apply(Lemmatize)

In [40]:
Req_data.head()

,gender,academics,title,category,post_type
0,female,undergraduate,save earth,visual art graphic design artistic design grap...,artwork
1,male,undergraduate,save earth,visual art graphic design artistic design grap...,artwork
2,female,graduate,save earth,visual art graphic design artistic design grap...,artwork
3,male,graduate,save earth,visual art graphic design artistic design grap...,artwork
4,male,undergraduate,save earth,visual art graphic design artistic design grap...,artwork


In [41]:
Req_data = Req_data.drop(["gender","academics"],axis=1)
Req_data.to_csv("Req_Data.csv")

In [42]:
corpus= []

for sents in Req_data["category"]:
  sents = nltk.word_tokenize(sents)
  if sents not in corpus:
    corpus.extend(sents)

for sents in Req_data["title"]:
  sents = nltk.word_tokenize(sents)
  if sents not in corpus:
    corpus.extend(sents)
    
corpus = set(corpus)
corpus = list(corpus)

In [49]:
import nltk 
from nltk.corpus import wordnet 

def recommend(a):
  def get_sim_words(w1):
    sim_words = []
    req_words = []
    for i in corpus:
      try:
        w2 = i+".n.01"
        w2 = wordnet.synset(w2)
        if w1.wup_similarity(w2) > 0.8:
          req_words.append(w2)
      except:
        try:
          w2 = i+".v.01"
          w2 = wordnet.synset(w2)
          if w1.wup_similarity(w2) > 0.8:
            req_words.append(w2)
        except:
          pass

    for i in req_words:
      for j in i.lemmas():
        sim_words.append(j.name())

    return sim_words

  w1 = a

  try:
    try:  
      w1 = w1+".n.01"
      w1 = wordnet.synset(w1)
      sim_words = get_sim_words(w1)
    except:
      w1 = w1+".v.01"
      w1 = wordnet.synset(w1)
      sim_words = get_sim_words(w1)
  except:
    words = w1.split(".")
    sim_words = [words[0].lower()]

  frames = []
  for i in sim_words:
    frames.append(Req_data[Req_data.apply(lambda row: row.astype(str).str.contains(i).any(), axis=1)])

  result = pd.concat(frames)

  return result.sample(frac = 1)

In [50]:
a = input("Enter a word for Recommendation: ")
recommend(a)

#This will return all the recommended posts by taking the meaning of the words.

Enter a word for Recommendation: learn


,title,category,post_type
355,gan part,computer technology machine learning,blog
867,ml ai,artificial intelligence machine learning infor...,blog
353,gan part,computer technology machine learning,blog
354,gan part,computer technology machine learning,blog
1031,always right,learning,skill
...,...,...,...
347,recommend system machine learning,computer technology machine learning,blog
255,face recognition using transfer learning,science technology,skill
1030,always right,learning,skill
549,keen learner,photography,skill
